In [1]:
!ls

adc.json  data_analysis  drive	sample_data


在使用Colaboratory访问谷歌云盘时，必须先进行授权操作。
授权绑定代码如下

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#指定当前工作路径
import os
os.chdir("drive/Colab Notebooks/DataWhale")

作业一：

任务1 - 数据分析（2天）,任务1打卡链接提交截止日期为20190512晚上10点
数据集下载
这是我们本次实践数据的下载地址 https://pan.baidu.com/s/1wO9qJRjnrm8uhaSP67K0lw
说明：这份数据集是金融数据（非原始数据，已经处理过了），我们要做的是预测贷款用户是否会逾期。表格中 "status" 是结果标签：0表示未逾期，1表示逾期。
数据集涉密，不要开源到网上，谢谢~

要求：数据切分方式 - 三七分，其中测试集30%，训练集70%，随机种子设置为2018

任务1：对数据进行探索和分析。时间：2天
数据类型的分析，无关特征删除，数据类型转换，缺失值处理，……以及你能想到和借鉴的数据分析处理

本作业从原始数据入手，从分析思路上讲，拿到数据后主要有以下几个步骤处理：

1，看其各个列的信息，包括其属性、缺失值、类别等信息,。统计缺失值数目，数据类型是否需要变换，这里可以写一个缺失值检测函数，以便快捷的检测整个数据的缺失值，唯一值等信息

2，根据列的信息对列进行筛选，将缺失值较多、唯一值百分比为1、字符型数据删除

3，将分类信息的列由类别转换为数值型的数据

4，对存在的缺失值进行填充，如利用均值进行填充等

5，在前处理完成后，对已有的数据进行标准化

6，再处理完数据后，对数据进行切割，分为训练集和测试集

7，选择合适的模型进行训练

8，对训练结果进行验证

In [0]:
#1 加载分析需要的包
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [6]:
#2 读取数据
data1 = pd.read_csv("data.csv",index_col=0,encoding="gbk")
data1.info()
print(data1.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4754 entries, 5 to 11992
Data columns (total 89 columns):
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility                             4752 non-null float64
student_featur

In [7]:
#3 编写函数来统计数据中每一列的缺失值情况，唯一值数量，唯一率信息等
def desc(df):
    loss_num=[] #缺失值的数量
    valid_num=[] #非缺失值的数量
    unique_num=[] #唯一的数量
    unique_percent=[] #唯一率
    for i,j in enumerate (df.columns): # enumerator 函数可以同时获得索引和值。 i表示索引，j表示值
        loss_num.append(df[df[j].isnull()].shape[0]) #isnull() 表示的是空 返回的是True（空）
        valid_num.append(df[df[j].notnull()].shape[0])
        unique_num.append(len(df[j].unique()))
        unique_percent.append(len(df[df[j].notnull()][j].unique())/df[df[j].notnull()].shape[0])
    data_desc = pd.DataFrame({"loss_num":loss_num,"valid_num":valid_num,"unique_num":unique_num,"unique_percent":unique_percent},index=df.columns)
    return data_desc
data_count = desc(data1)
data_count

,loss_num,valid_num,unique_num,unique_percent
custid,0,4754,4754,1.000000
trade_no,0,4754,4754,1.000000
bank_card_no,0,4754,1,0.000210
low_volume_percent,2,4752,41,0.008418
middle_volume_percent,2,4752,91,0.018939
take_amount_in_later_12_month_highest,0,4754,166,0.034918
trans_amount_increase_rate_lately,3,4751,783,0.164597
trans_activity_month,2,4752,85,0.017677
trans_activity_day,2,4752,513,0.107744
transd_mcc,2,4752,42,0.008628


In [8]:
#4 统计每列的数据类型，根据类型对数据进行描述统计
data1.dtypes.value_counts()
#选择列
num_features = data1.select_dtypes(include = ["float64","int64"]).copy()
obj_features = data1.select_dtypes(include=['object']).copy()
obj_features_desc = desc(obj_features)
obj_features_desc

,loss_num,valid_num,unique_num,unique_percent
trade_no,0,4754,4754,1.000000
bank_card_no,0,4754,1,0.000210
reg_preference_for_trad,2,4752,6,0.001052
source,0,4754,1,0.000210
id_name,276,4478,4310,0.962260
latest_query_time,304,4450,208,0.046517
loans_latest_time,297,4457,233,0.052053


In [12]:
#5 根据结果对字符型的列进行删除或数值转换
data1.drop(columns=['latest_query_time','loans_latest_time','trade_no','id_name','bank_card_no','source'],axis=1,inplace=True)
data1['reg_preference_for_trad'].replace({'一线城市':1,'二线城市':2,'三线城市':3,'境外':4,'其他城市':5},inplace = True)
#由于reg_preference_for_trad这一列有缺失数据，需要根据情况对其进行填充，我们将reg_preference_for_trad数据缺额全部填充为其他城市
data['reg_preference_for_trad'].fillna("其他城市",inplace=True)

KeyError: ignored

In [0]:
#6 对数值型的列进行处理，根据描述统计的结果进行删减
num_features_desc = desc(num_features)
num_features_desc
num_features.drop(columns=['custid','student_feature'],axis=1,inplace=True)

In [0]:
data1.drop(columns=['custid','student_feature'],axis=1,inplace=True)

In [0]:
#7 将数据分为训练集和标签
df_1,Y = data1.drop(columns=['status'],axis=1),data1['status']

In [18]:
#8 对数值型的数据进行缺失值填充，利用均值进行填充
imputer = Imputer(strategy='median')
df_2 = imputer.fit_transform(df_1)
df_2 = pd.DataFrame(df_2,columns=df_1.columns)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
df_2_desc=desc(df_2)
df_2_desc

,loss_num,valid_num,unique_num,unique_percent
low_volume_percent,0,4754,40,0.008414
middle_volume_percent,0,4754,90,0.018931
take_amount_in_later_12_month_highest,0,4754,166,0.034918
trans_amount_increase_rate_lately,0,4754,782,0.164493
trans_activity_month,0,4754,84,0.017669
trans_activity_day,0,4754,512,0.107699
transd_mcc,0,4754,41,0.008624
trans_days_interval_filter,0,4754,147,0.030921
trans_days_interval,0,4754,114,0.023980
regional_mobility,0,4754,5,0.001052


In [20]:
#9 数据标准化 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_3 = scaler.fit_transform(df_2) # 生成结果是array类型，需要转化为df类型
df_3 = pd.DataFrame(df_3,columns=df_2.columns)
df_3.head()

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,-0.284251,0.612374,-0.494499,-0.019099,-1.292452,-0.308087,-0.112363,-0.088968,0.258090,0.360860,...,-0.330249,-0.233181,-1.104758,-0.087098,-1.277708,-0.973381,-1.068511,-1.226123,-0.313460,-0.692110
1,-0.043372,0.267106,0.015242,-0.018551,0.993418,0.544138,0.334643,0.043178,-0.470508,1.484325,...,0.089632,-0.109253,0.840507,0.267713,0.108626,0.238311,0.490237,0.256592,-0.532163,-0.999858
2,0.438384,0.405213,-0.494499,-0.018955,0.993418,-1.477693,-1.006374,1.717020,0.015224,-1.886068,...,-1.239989,-1.008619,-0.684917,0.764448,-1.277708,-1.276304,-0.852018,-0.681007,-0.586839,-0.922921
3,-0.525129,0.405213,0.015242,-0.020208,-1.190857,2.419034,1.005152,-0.661598,-0.956239,0.360860,...,-0.120309,-0.493431,1.008443,0.267713,0.108626,-0.064612,1.009820,0.769001,-0.586839,-0.961389
4,-0.284251,0.612374,-0.494499,-0.019733,0.993418,-1.119171,-1.006374,1.628923,1.229553,-1.886068,...,-0.750129,-0.335865,-0.111133,0.196751,-0.931125,-0.973381,-0.548928,0.054899,-0.040081,1.269785


In [21]:
df_3.shape

(4754, 80)

In [22]:
#10 将数据切分为训练集和验证集
train_x,test_x,train_y,test_y = train_test_split(df_3,Y,train_size=0.7,test_size=0.3,random_state=1)
print(train_x.shape)
print(train_y.shape)

(3327, 80)
(3327,)


In [23]:
#11 使用svm训练模型
from sklearn.svm import SVC
clf = SVC()
clf.fit(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [24]:
#12 验证结果
pred_y = clf.predict(test_x)
clf.score(test_x,test_y)

0.7869656622284513